In [1]:
# Tensorflow Interactive Session 시작
import tensorflow as tf
sess = tf.InteractiveSession()

In [2]:
x = tf.placeholder(tf.float32, [100, 32, 32, 3])

# tf.layers.conv2d

![image.png](attachment:image.png)

conv2d(
    inputs,
    filters,
    kernel_size,
    strides=(1, 1),
    padding='valid',
    data_format='channels_last',
    dilation_rate=(1, 1),
    activation=None,
    use_bias=True,
    kernel_initializer=None,
    bias_initializer=tf.zeros_initializer(),
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    trainable=True,
    name=None,
    reuse=None
)

https://www.tensorflow.org/api_docs/python/tf/layers/conv2d

# Activations
- tf.nn.relu
- tf.tanh
- tf.sigmoid

# Kernel_initializers / Bias_initializers
- tf.zeros_initializer
- tf.ones_initializer
- tf.constant_initializer
- tf.random_uniform_initializer
- tf.random_normal_initializer
- tf.truncated_normal_initializer

In [3]:
net = tf.layers.conv2d(x, 
                       20, 
                       [5,5],  # indepth는 x에 있음. 알아서 채워준다.
                       strides=[2,2], 
                       padding='SAME', 
                       activation=tf.tanh,  # tf.tanh()가 아님에 주의
                       kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                       bias_initializer=tf.zeros_initializer(),
                       name='conv1')
net

<tf.Tensor 'conv1/Tanh:0' shape=(100, 16, 16, 20) dtype=float32>

<연습문제>
1. 아래와 같은 조건의  netx = tf.layers.conv2d(net, ...  를 선언해봅시다.
    - outdepth 30
    - kernel 크기는 [7,7]
    - strides는 [2,2]
    - padding은 'VALID'
    - activation은 ReLU
    - kernel_initialzer는 random_uniform
    - bias_initializer는 ones

In [4]:
netx = tf.layers.conv2d(x, 30, [7,7], strides=[2,2],
                        padding='VALID',activation=tf.nn.relu,
                        kernel_initializer=tf.random_uniform_initializer(),
                        bias_initializer=tf.ones_initializer(),
                        name='convx')

# Batch Normalization

In [5]:
is_training = tf.placeholder(tf.bool)

net = tf.layers.conv2d(x, 
                       20, 
                       [5,5], 
                       strides=[2,2], 
                       padding='SAME', 
                       #activation=tf.tanh,
                       activation=None,
                       kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                       bias_initializer=tf.zeros_initializer(),
                       name='conv2')
net = tf.layers.batch_normalization(net, training=is_training)
net = tf.tanh(net)
net

<tf.Tensor 'Tanh:0' shape=(100, 16, 16, 20) dtype=float32>

<연습문제>
1. 아래와 같은 조건의 convolutional layer 'netx'를 선언해봅시다.
    - batch normalization 적용
    - 이외 모든 조건은 위와 동일

In [6]:
is_training = tf.placeholder(tf.bool)

netx = tf.layers.conv2d(x, 30, [7,7], strides=[2,2],
                        padding='VALID',activation=tf.nn.relu,
                        kernel_initializer=tf.random_uniform_initializer(),
                        bias_initializer=tf.ones_initializer(),
                        name='convx2')
netx = tf.layers.batch_normalization(net, training=is_training)
netx = tf.tanh(netx)
netx

<tf.Tensor 'Tanh_1:0' shape=(100, 16, 16, 20) dtype=float32>

# Weight Decay

In [7]:
WEIGHT_DECAY = 0.0005
var_list = [v for v in tf.trainable_variables()]
l2_loss = tf.add_n([ tf.nn.l2_loss(v) for v in var_list
                     if 'kernel' in v.name ]) * WEIGHT_DECAY

<연습문제>
1. 'bias'의 l2_loss도 loss에 추가해봅시다. 참조: print(var_list)

In [8]:
print(var_list)
bias_l2_loss = tf.add_n([ tf.nn.l2_loss(v) for v in var_list
                            if 'bias' in v.name]) * WEIGHT_DECAY

[<tf.Variable 'conv1/kernel:0' shape=(5, 5, 3, 20) dtype=float32_ref>, <tf.Variable 'conv1/bias:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'convx/kernel:0' shape=(7, 7, 3, 30) dtype=float32_ref>, <tf.Variable 'convx/bias:0' shape=(30,) dtype=float32_ref>, <tf.Variable 'conv2/kernel:0' shape=(5, 5, 3, 20) dtype=float32_ref>, <tf.Variable 'conv2/bias:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'batch_normalization/beta:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'batch_normalization/gamma:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'convx2/kernel:0' shape=(7, 7, 3, 30) dtype=float32_ref>, <tf.Variable 'convx2/bias:0' shape=(30,) dtype=float32_ref>, <tf.Variable 'batch_normalization_1/beta:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'batch_normalization_1/gamma:0' shape=(20,) dtype=float32_ref>]


# tf.layers.max_pooling2d

max_pooling2d(
    inputs,
    pool_size,
    strides,
    padding='valid',
    data_format='channels_last',
    name=None
)

https://www.tensorflow.org/api_docs/python/tf/layers/max_pooling2d

In [9]:
net = tf.layers.max_pooling2d(net,
                              [3,3],
                              strides=[2,2],
                              padding='VALID',
                              name='pool1')
net

<tf.Tensor 'pool1/MaxPool:0' shape=(100, 7, 7, 20) dtype=float32>

<연습문제>
1. kernel [2,2], stride [2,2], padding='VALID'의 max pooling layer를 선언해봅시다. (netx = ...)

In [10]:
netx = tf.layers.max_pooling2d(netx, [2,2], strides=[2,2],
                              padding='VALID', name='poolx1')
netx

<tf.Tensor 'poolx1/MaxPool:0' shape=(100, 8, 8, 20) dtype=float32>

In [11]:
net = tf.reshape(net, [-1, 7*7*20])
net

<tf.Tensor 'Reshape:0' shape=(100, 980) dtype=float32>

# tf.layers.dense

dense(
    inputs,
    units,
    activation=None,
    use_bias=True,
    kernel_initializer=None,
    bias_initializer=tf.zeros_initializer(),
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    trainable=True,
    name=None,
    reuse=None
)

https://www.tensorflow.org/api_docs/python/tf/layers/dense

In [12]:
net = tf.layers.dense(net,
                      128,
                      activation=tf.nn.tanh,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                      bias_initializer=tf.constant_initializer(0.1),
                      name='fc3')
net

<tf.Tensor 'fc3/Tanh:0' shape=(100, 128) dtype=float32>

<연습문제>
1. 아래와 같은 조건의  netx = tf.layers.dense(net, ...  를 선언해봅시다.
    - outdepth 10
    - activation은 ReLU
    - kernel_initialzer는 truncated_normal
    - bias_initializer는 constant 0.2

In [13]:
netx = tf.layers.dense(netx, 10, activation=tf.nn.relu,
                       kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                       bias_initializer=tf.constant_initializer(0.2),
                       name='fcx3')

netx

<tf.Tensor 'fcx3/Relu:0' shape=(100, 8, 8, 10) dtype=float32>

# tf.layers.dropout

dropout(
    inputs,
    rate=0.5,
    noise_shape=None,
    seed=None,
    training=False,
    name=None
)

https://www.tensorflow.org/api_docs/python/tf/layers/dropout

In [14]:
is_training = tf.placeholder(tf.bool)

net = tf.layers.dropout(net,
                        rate=0.5,
                        training=is_training,
                        name='drop3')
net

<tf.Tensor 'drop3/cond/Merge:0' shape=(100, 128) dtype=float32>

# model save/restore

In [22]:
NUM_EPOCHS = 1
CHECKPOINT_PATH = './checkpoints'

import os
if not os.path.isdir(CHECKPOINT_PATH):
  os.makedirs(CHECKPOINT_PATH)

In [23]:
# Initialize an saver for store model checkpoints
saver = tf.train.Saver()

tf.train.Saver.save(
    sess,
    save_path,
    global_step=None,
    latest_filename=None,
    meta_graph_suffix='meta',
    write_meta_graph=True,
    write_state=True
)

In [24]:
global_step = tf.Variable(0, trainable=False, dtype=tf.float32, name='global_step')

In [25]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, gpu_options={'allow_growth':True}))

In [26]:
init_op = tf.group(tf.global_variables_initializer(),
                 tf.local_variables_initializer())
sess.run(init_op)

In [27]:
for epoch in range(0, NUM_EPOCHS):
    checkpoint_name = os.path.join(CHECKPOINT_PATH,
                        'example_ep-'+str(epoch+1)+'_step')
    save_path = saver.save(sess, checkpoint_name, global_step=global_step)
    print("Epoch: %d, Model checkpoint saved at %s" % (epoch+1, save_path))

Epoch: 1, Model checkpoint saved at ./checkpoints/example_ep-1_step-0


tf.train.Saver.restore(
    sess,
    save_path
)

In [21]:
latest_ckpt = tf.train.latest_checkpoint(CHECKPOINT_PATH)
print("Loading the last checkpoint: " + latest_ckpt)
saver.restore(sess, latest_ckpt)

Loading the last checkpoint: ./checkpoints/example_ep-1_step-0
INFO:tensorflow:Restoring parameters from ./checkpoints/example_ep-1_step-0


# TFRecords input pipelines ...